In [12]:
!git clone https://github.com/Dt-Pham/Advanced-Lane-Lines.git
%cd Advanced-Lane-Lines/

Cloning into 'Advanced-Lane-Lines'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 104 (delta 3), reused 0 (delta 0), pack-reused 99 (from 1)
Receiving objects: 100% (104/104), 95.77 MiB | 35.35 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/Advanced-Lane-Lines/Advanced-Lane-Lines


In [16]:
import sys
import subprocess
import importlib

# -----------------------------
# Auto-install ultralytics if missing
# -----------------------------
package_name = 'ultralytics'
try:
    import ultralytics
except ModuleNotFoundError:
    print(f"{package_name} not found. Installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name, "--upgrade"])
finally:
    globals()[package_name] = importlib.import_module(package_name)
    from ultralytics import YOLO

# -----------------------------
# Other imports
# -----------------------------
import numpy as np
import matplotlib.image as mpimg
import cv2
from moviepy.editor import VideoFileClip

# Temporary fix for deprecated np.int
np.int = int

# Import your existing modules
from CameraCalibration import CameraCalibration
from Thresholding import *
from PerspectiveTransformation import *
from LaneLines import *

# -----------------------------
# Object Detection Class
# -----------------------------
class ObjectDetection:
    def __init__(self, model_path="yolov8n.pt"):  # tiny YOLOv8 model
        self.model = YOLO(model_path)

    def forward(self, img):
        results = self.model.predict(img, verbose=False)[0]
        annotated_img = results.plot()  # draw bounding boxes
        return annotated_img

# -----------------------------
# Combined Lane + Object Detection
# -----------------------------
class LaneAndObjectDetection:
    def __init__(self):
        self.calibration = CameraCalibration('camera_cal', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()
        self.detector = ObjectDetection("yolov8n.pt")  # YOLOv8 model

    def forward(self, img):
        out_img = np.copy(img)

        # -----------------------------
        # Lane Detection
        # -----------------------------
        img_lane = self.calibration.undistort(img)
        img_lane = self.transform.forward(img_lane)
        img_lane = self.thresholding.forward(img_lane)
        img_lane = self.lanelines.forward(img_lane)
        img_lane = self.transform.backward(img_lane)
        out_img = cv2.addWeighted(out_img, 1, img_lane, 0.6, 0)
        out_img = self.lanelines.plot(out_img)

        # -----------------------------
        # Object Detection
        # -----------------------------
        out_img = self.detector.forward(out_img)

        return out_img

    # -----------------------------
    # Video Processing
    # -----------------------------
    def process_video(self, input_path, output_path):
        clip = VideoFileClip(input_path)
        out_clip = clip.fl_image(self.forward)
        out_clip.write_videofile(output_path, audio=False)

    # -----------------------------
    # Image Processing
    # -----------------------------
    def process_image(self, input_path, output_path):
        img = mpimg.imread(input_path)
        out_img = self.forward(img)
        mpimg.imsave(output_path, out_img)

# -----------------------------
# Main
# -----------------------------
def main():
    detector = LaneAndObjectDetection()
    # Update input/output paths
    input_video = "challenge_video.mp4"
    output_video = "output_with_objects.mp4"
    detector.process_video(input_video, output_video)

    # Optional: process a single image
    # detector.process_image("test_image.jpg", "output_image.jpg")

if __name__ == "__main__":
    main()


ultralytics not found. Installing...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Moviepy - Building video output_with_objects.mp4.
Moviepy - Writing video output_with_objects.mp4



t: 100%|██████████| 485/485 [01:44<00:00,  4.06it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file challenge_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame 484/485, at time 16.15/16.15 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready output_with_objects.mp4
